In [1]:

import scipy.io
import torch
import importlib
import matplotlib.pyplot as plt
import numpy as np

import src.HyperParameters as hp 
import src.data as dt 
import src.model3 as Model 
import src.train as trainer

importlib.reload(hp)
importlib.reload(dt)
importlib.reload(Model)
importlib.reload(trainer)



<module 'src.train' from '/Users/ramtarun/Desktop/Cambridge/Friction-Factor-Estimation-PINN/src/train.py'>

In [2]:
def list_available_devices():
    devices = []
    if torch.cuda.is_available():
        devices.append('cuda')
    if torch.cuda.is_available() and torch.cuda.device_count() > 1:
        devices.append('multi-gpu')
    if torch.cuda.is_available() and torch.cuda.nccl.version() > 0:
        devices.append('nccl')
    if torch.cuda.is_available() and torch.cuda.is_initialized():
        devices.append('initialized')
    if torch.cuda.is_available() and torch.cuda.memory_allocated() > 0:
        devices.append('memory-allocated')
    if torch.cuda.is_available() and torch.cuda.memory_reserved() > 0:
        devices.append('memory-reserved')
    devices.append('cpu')
    devices.append('mps')

    return devices

def get_device_choice():
    devices = list_available_devices()
    print("Available devices:")
    for i, device in enumerate(devices):
        print(f"{i + 1}. {device}")
    while True:
        choice = input("Choose the device for model training (enter the corresponding number): ")
        if choice.isdigit() and 1 <= int(choice) <= len(devices):
            return devices[int(choice) - 1]
        print("Invalid choice. Please enter a valid device number.")

# Usage
device = get_device_choice()
print(f"Selected device: {device}")

Available devices:
1. cpu
2. mps
Selected device: cpu


In [3]:
data = scipy.io.loadmat('/Users/ramtarun/Desktop/Cambridge/Indirect-Noise-in-Nozzles/Data/Data_PINN_subsonic_geom_linvelsup_f0-0.1.mat')

In [4]:
PINN_model = Model.PINN(hp.input_size, hp.output_size, hp.hidden_size, hp.num_layers, hp.lda)
PINN_model.to(device, non_blocking=False)

PINN(
  (loss_function): MSELoss()
  (rnn): PhyGRU(
    (gru): GRU(2, 4, num_layers=2, batch_first=True, dropout=0.5)
    (output_layer): Linear(in_features=4, out_features=4, bias=True)
    (activation): Tanh()
  )
)

In [ ]:
params = list(PINN_model.parameters())
# optimizer = torch.optim.Adam([{'params' : params[1::]},{'params' : params[-1], 'lr': hp.ff_learning_rate}], lr = hp.learning_rate, amsgrad = True)   
optimizer = torch.optim.Adam(params=params, lr = hp.learning_rate, amsgrad = True)   
# optimizer = torch.optim.LBFGS(params, hp.ff_learning_rate, 
#                               max_iter = hp.epochs, 
#                               max_eval = None, 
#                               tolerance_grad = 1e-11, 
#                               tolerance_change = 1e-11, 
#                               history_size = 100, 
#                               line_search_fn = 'strong_wolfe')


In [ ]:
device

In [ ]:
inputs, targets, meanflow =  dt.DataPreprocessing(data, ff=0.01, device = device)
inputs.to(device)
targets.to(device)
meanflow.to(device)

N = inputs.shape[1]
train_loader, val_loader = dt.DataTransformer(inputs, targets, meanflow, TrainingSet=True)

meanflow.device

In [ ]:
fval = [0.1  , 0.06 , 0.04, 0.01]

# for f in fval:
# ### Model 3
# batch_size = 32
# sequence_length = len(inputs) // batch_size
# # Reshape the input tensors within the DataLoader
# train_loader = []
# for batch in Train_loader:
#     input_data = batch.view(batch_size, sequence_length, 1)
#     train_loader.append(input_data)

train_loss, val_loss, f_train, f_test, f_dist = trainer.train(train_loader, val_loader, hp.epochs, optimizer, PINN_model, N)

with torch.no_grad():
    plt.figure() 
    plt.plot(train_loss.keys(), train_loss.values(), 'r-', label='Training Loss')
    plt.plot(val_loss.keys(), val_loss.values(), 'g-', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()
    plt.figure()
    plt.plot(f_train.keys(), f_train.values(), 'r-', label='Training Friction Factor')
    plt.plot(f_test.keys(), f_test.values(), 'g-', label='Validation Friction Factor')
    plt.xlabel('Epochs')
    plt.ylabel('Friction Factor')
    plt.legend()
    plt.grid()
    plt.show()
    plt.figure()
